## Description

This notebook provides a user interface to interactively select a region for extraction of EvapoTranspiration data calculated using the geeSEBAL library.  The SEBAL calculations for ET are leveraged from https://github.com/gee-hydro/geeSEBAL

The map displays labeled sample locations.  
- Irrigated locations are shown in Blue.  
- Rainfed locations are shown in Red.

Usage:

    1) User selects an area of interest on the map
    2) User specifies a name for the area to use in the output filename
    3) Sample locations are generated within the selected area
    4) Batch data exports are launched for the sample locations
    5) Data is saved on user's Google Drive (default path is root/irrigation/)

## Setup Notebook

#### Setup Google Earth Engine libraries

This notebook requires the installation of:
- earthengine
- geemap
- geopandas

In [1]:
#!pip install earthengine-api
#!pip install geemap
#!pip install geopandas

In [ ]:
import ee
#ee.Authenticate() # Uncomment this line when first running the notebook in a new environment
ee.Initialize()
import geemap

#### Setup custom libraries

In [10]:
import boundaries as bds
import labeledsamples as ls
import dataextract as dx





#### Setup Defined Regions of Interest

In [8]:
irr_west, rain_west, irr_east, rain_east = ls.getRawLabeledData()

Map = geemap.Map()
Map.add_basemap('HYBRID')
Map.addLayer(irr_west, { 'color': 'blue'}, 'irrigated US-West', opacity=0.5)
Map.addLayer(rain_west, { 'color': 'red'}, 'rainfed US-West', opacity=0.5)
Map.addLayer(bds.ca_rainfed, { 'color': 'red'}, 'rainfed US-CA', opacity=0.5)
Map.addLayer(bds.or_rainfed, { 'color': 'red'}, 'rainfed US-OR', opacity=0.5)
Map.addLayer(irr_east, { 'color': 'blue'}, 'irrigated US-East', opacity=0.5)
Map.addLayer(rain_east, { 'color': 'red'}, 'rainfed US-East', opacity=0.5)

## 1) Manually select a region for data extraction

Draw a bounding box or polygon on the map

In [ ]:
Map

## 2) Specify the name of the area of interest

In [ ]:
aoi_name = 'wesus15_WI' # to be manually specified

## 3) Generate sample locations within the selected region

In [17]:
if Map and (len(Map.draw_features) > 0):
    AOI = ee.FeatureCollection(Map.draw_features)
    sample_locs = ls.generateAnnualSampleLocations(AOI, aoi_name, num_samples=50, year=None)
    
    # update the prior map, to display the generated locations
    Map.addLayer(sample_locs.draw(color="yellow", pointRadius=5, strokeWidth=3), {}, 'sample_locations')
    Map.centerObject(sample_locs)

    print('Number of sample locations by type (0=rainfed, 1=irrigated)')
    print(sample_locs.aggregate_histogram('POINT_TYPE').getInfo())

# sample locations 100


## 4) Launch the extract processes in batch mode

In [ ]:
if sample_locs:
  et = dx.extractData(sample_locs, aoi_name, model='')

# locations 94


## 5) Retrieve the count of batch jobs in-progress

This will be zero at the end of the batch run

In [ ]:
!earthengine task list --status RUNNING READY | wc -l